In [1]:
!pip install numpy matplotlib

!pip install ipython

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [2]:
from quantum_gate_package import I, X,P0, P1, H, Z,Y

Shor Code [[9,1,3]] simulation tasks for [{

The Shor code corrects any single-qubit Pauli error.

Phase error syndrome operators:

\[
S_1 = X_1X_2X_3X_4X_5X_6,\quad
S_2 = X_4X_5X_6X_7X_8X_9.
\]

We test that a phase-flip \(Z_i\) anticommutes with the stabilizers containing \(X_i\).

to do:
1. Build S1 and S2 as 9-qubit operators
2. For each i=1..9:
3. build Zi
4. check commutator: S*Zi == Zi*S (commute) or S*Zi == -Zi*S (anticommute)
5. print a table of which Zi flips which stabilizer


In [3]:
import numpy as np

def kron(*ops):
    out = np.array([[1]], dtype=complex)
    for op in ops:
        out = np.kron(out, op)
    return out

def fidelity(psi_ref, psi_test):
    return float(np.abs(np.vdot(psi_ref, psi_test))**2)

# ================================
# Operators on 3 qubits (|q2 q1 q0>)
# ================================
def H_on_qubit(i):
    if i == 0:
        return kron(I, I, H)
    if i == 1:
        return kron(I, H, I)
    if i == 2:
        return kron(H, I, I)

def X_on_qubit(i):
    if i == 0:
        return kron(I, I, X)
    if i == 1:
        return kron(I, X, I)
    if i == 2:
        return kron(X, I, I)

def Z_on_qubit(i):
    if i == 0:
        return kron(I, I, Z)
    if i == 1:
        return kron(I, Z, I)
    if i == 2:
        return kron(Z, I, I)

# ================================
# Phase-flip code encoding
# Encode into |+++> and |--->
# Equivalent: apply H⊗3 then do bitflip encoding then H⊗3
#
# We will encode directly:
# |psi> = alpha|0> + beta|1>
# -> alpha|+++> + beta|--->
# ================================
def encode_phaseflip3(alpha, beta):
    # |+> and |->
    plus  = (1/np.sqrt(2)) * np.array([1, 1], dtype=complex)
    minus = (1/np.sqrt(2)) * np.array([1,-1], dtype=complex)

    ket_ppp = np.kron(np.kron(plus, plus), plus)
    ket_mmm = np.kron(np.kron(minus, minus), minus)

    return alpha * ket_ppp + beta * ket_mmm

# ================================
# X stabilizers for phase-flip code:
# X1X2 and X2X3   (in q0,q1,q2 -> careful ordering)
# In |q2 q1 q0> ordering:
# X1X2 means X(q0)X(q1) = I ⊗ X ⊗ X
# X2X3 means X(q1)X(q2) = X ⊗ X ⊗ I
# ================================
X1X2 = kron(I, X, X)
X2X3 = kron(X, X, I)

def syndrome_phaseflip(state):
    s1 = np.real(np.vdot(state, X1X2 @ state))
    s2 = np.real(np.vdot(state, X2X3 @ state))
    s1 = 1 if s1 >= 0 else -1
    s2 = 1 if s2 >= 0 else -1
    return (s1, s2)

def correction_from_syndrome_phaseflip(s):
    # Same table form but now means Z error location:
    # (+1,+1) -> no error
    # (-1,+1) -> Z on q0
    # (-1,-1) -> Z on q1
    # (+1,-1) -> Z on q2
    if s == (1, 1):
        return None
    if s == (-1, 1):
        return 0
    if s == (-1, -1):
        return 1
    if s == (1, -1):
        return 2

def correct_phaseflip(state):
    s = syndrome_phaseflip(state)
    q = correction_from_syndrome_phaseflip(s)
    if q is None:
        return state, s, None
    corrected = Z_on_qubit(q) @ state
    return corrected, s, q

# ================================
# ✅ DEMO (single Z error correction)
# ================================
alpha = 0.6
beta = np.sqrt(1 - alpha**2)

psi_L = encode_phaseflip3(alpha, beta)

print("Syndrome (no error):", syndrome_phaseflip(psi_L))

for err in [None, 0, 1, 2]:
    # apply Z error on qubit err
    noisy = psi_L.copy()
    if err is not None:
        noisy = Z_on_qubit(err) @ noisy

    corrected, syn, qcorr = correct_phaseflip(noisy)
    F = fidelity(psi_L, corrected)

    print("\n----------------------------")
    print("Z error applied on qubit:", err)
    print("Measured syndrome (X1X2, X2X3):", syn)
    print("Correction applied on qubit:", qcorr)
    print("Fidelity after correction:", F)

Syndrome (no error): (1, 1)

----------------------------
Z error applied on qubit: None
Measured syndrome (X1X2, X2X3): (1, 1)
Correction applied on qubit: None
Fidelity after correction: 0.9999999999999993

----------------------------
Z error applied on qubit: 0
Measured syndrome (X1X2, X2X3): (-1, 1)
Correction applied on qubit: 0
Fidelity after correction: 0.9999999999999993

----------------------------
Z error applied on qubit: 1
Measured syndrome (X1X2, X2X3): (-1, -1)
Correction applied on qubit: 1
Fidelity after correction: 0.9999999999999993

----------------------------
Z error applied on qubit: 2
Measured syndrome (X1X2, X2X3): (1, -1)
Correction applied on qubit: 2
Fidelity after correction: 0.9999999999999993
